In [ ]:
!pip install -q google-generativeai google-ai-generativelanguage

In [ ]:
import os
import google.generativeai as genai
from google.ai.generativelanguage_v1beta.types import content
from google.colab import userdata

genai.configure(api_key=userdata.get('GOOGLE_API_KEY'))

def multiply_two_values(value1: int, value2: int):
    """returns value1 * value2."""
    return value1*value2

tools = [multiply_two_values]

model = genai.GenerativeModel(model_name='gemini-2.0-flash-exp',
                              tools=tools)

In [ ]:
model._tools.to_proto()

[function_declarations {
   name: "multiply_two_values"
   description: "returns value1 * value2."
   parameters {
     type_: OBJECT
     properties {
       key: "value2"
       value {
         type_: INTEGER
       }
     }
     properties {
       key: "value1"
       value {
         type_: INTEGER
       }
     }
     required: "value1"
     required: "value2"
   }
 }]

In [ ]:
chat = model.start_chat()
response = chat.send_message('Can you multiply 2414141 x 4125121')

In [ ]:
response

response:
GenerateContentResponse(
    done=True,
    iterator=None,
    result=protos.GenerateContentResponse({
      "candidates": [
        {
          "content": {
            "parts": [
              {
                "function_call": {
                  "name": "multiply_two_values",
                  "args": {
                    "value1": 2414141.0,
                    "value2": 4125121.0
                  }
                }
              }
            ],
            "role": "model"
          },
          "finish_reason": "STOP",
          "safety_ratings": [
            {
              "category": "HARM_CATEGORY_HATE_SPEECH",
              "probability": "NEGLIGIBLE"
            },
            {
              "category": "HARM_CATEGORY_DANGEROUS_CONTENT",
              "probability": "NEGLIGIBLE"
            },
            {
              "category": "HARM_CATEGORY_HARASSMENT",
              "probability": "NEGLIGIBLE"
            },
            {
              "category": "H

In [ ]:
for part in response.parts:
    if fn := part.function_call:
        args = ", ".join(f"{key}={val}" for key, val in fn.args.items())
        print(f"{fn.name}({args})")

multiply_two_values(value1=2414141.0, value2=4125121.0)


In [ ]:
fc = response.candidates[0].content.parts[0].function_call

In [ ]:
fc

name: "multiply_two_values"
args {
  fields {
    key: "value2"
    value {
      number_value: 4125121
    }
  }
  fields {
    key: "value1"
    value {
      number_value: 2414141
    }
  }
}

In [ ]:
result = multiply_two_values(fc.args['value1'], fc.args['value2'])

In [ ]:
response = chat.send_message(
    genai.protos.Content(
    parts=[genai.protos.Part(
        function_response = genai.protos.FunctionResponse(
          name='multiply_two_values',
          response={'result': result}))]))

In [ ]:
response.text.strip()

'9958623736061'